In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import datetime
import os
import re
import time
import json
import subprocess
import glob
import calendar

In [2]:
CLOUD_GATE = r'https://echizen.cloudgate.jp/user-hub/yachiyo-ind/'

GMAIL_URL = r'https://mail.google.com/a/yachiyo-ind.co.jp/'
CALENDAR_URL = r'https://calendar.google.com/calendar/u/0/r?tab=mc&pli=1'
KOBETSU_GENKA_URL = r'https://echizen.cloudgate.jp/sso/yachiyo-ind/post.xhtml?providerUuid=284c5466-7256-49e3-99ab-04ff1a8178a8'
KINTAIKANRI_URL = r'https://yachiyo-ind.obic7.obicnet.ne.jp/Obic7FederationService'
GUEST_WIFI_URL = r'https://sites.google.com/yachiyo-ind.co.jp/yachiyo-portal/guest-wifi'

In [3]:

def start_google_chrome_with_port(url):
    
    chromePath = r'"C:\Program Files\Google\Chrome\Application\chrome.exe" -remote-debugging-port=9222 --user-data-dir=C:\Temp_Chrome'
    
    subprocess.Popen(chromePath)
    time.sleep(3)

    options = ChromeOptions()
    options.use_chromium = True
    options.debugger_address = r'127.0.0.1:9222'
    
    try:
        chromedriverPath = ChromeDriverManager().install()
    except:
        chromedriverPath = 'C:\python\chromedriver\chromedriver.exe'
    
    service = Service(executable_path=chromedriverPath)
    driver = Chrome(service=service, options=options)
    actionChains = ActionChains(driver)
    
    # driver.set_window_position(0, 0)
    driver.set_window_size(1300, 1000)
    driver.implicitly_wait(5)
    
    driver.get(url)
    
    return driver, actionChains



In [4]:
def send_keys_and_enter(driver, loginInfo, css, keys):
    
    wait = WebDriverWait(driver, 10)
    wait.until(visibility_of_all_elements_located((By.CSS_SELECTOR, css)))
    
    eles = driver.find_elements(By.CSS_SELECTOR, css)
    if eles:
        eles[0].clear()
        eles[0].send_keys(loginInfo[keys])
        eles[0].send_keys(Keys.ENTER)
        time.sleep(3)


def open_new_window(driver, url, css, winNum):
    
    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[winNum])
    driver.get(url)
    
    if url == GMAIL_URL:
        try:
            eles = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > button > span')
            if eles:
                eles[0].click()
        except:
            pass
        
    wait = WebDriverWait(driver, 30)
    wait.until(visibility_of_all_elements_located((By.CSS_SELECTOR, css)))
        
    if url == KOBETSU_GENKA_URL:
        eles = driver.find_elements(By.CSS_SELECTOR, '#bodyMain > header > nav > a')
        if eles:
            eles[0].click()
        eles = driver.find_elements(By.CSS_SELECTOR, 'section > ul > li:nth-child(2) > a')
        if eles:
            eles[0].click()

In [5]:
try:
    with open(glob.glob('../json/*.json')[0]) as f:
        loginInfo = json.load(f)
except FileNotFoundError as e:
    print(e)

In [6]:
driver, actionChains = start_google_chrome_with_port(CLOUD_GATE)

In [7]:
try:
    send_keys_and_enter(driver, loginInfo, '#rawUsername', 'ID')
    send_keys_and_enter(driver, loginInfo, '#password', 'PASSWORD')
except:
    pass

In [8]:
newWindosList = {
    # GMAIL_URL: '#\:mr > div > div > a',
    # CALENDAR_URL: 'body',
    KOBETSU_GENKA_URL: 'body',
    # KINTAIKANRI_URL: 'body',
}

In [9]:
for winNum, (url, css) in enumerate(newWindosList.items()):
    open_new_window(driver, url, css, winNum+1) 

In [61]:
# 日別合計
def extact_daily_total(rowNum):
    
    get_table = []
    
    css = f'#tblDayTotal > tbody > tr:nth-child({rowNum}) > td'
    for ele in driver.find_elements(By.CSS_SELECTOR, css):
        txt = ele.text
        if txt == '':
            get_table.append(0.0)
        else:
            get_table.append(txt)
            
    css = '#tblDayTotal_wrapper > div:nth-child(3) > div > div > div > div > table > thead > tr'
    index = ['index'] + driver.find_elements(By.CSS_SELECTOR, css)[0].text.split()
    
    return get_table, index

In [62]:
get_tables = []
for rowNum in range(5):
    get_table, columns = extact_daily_total(rowNum+1)
    get_tables.append(get_table)

In [81]:
df = pd.DataFrame(get_tables).T
df.columns = [df.iloc[0]]
df.index = columns
df2 = df.drop(index=df.index[[0, 1]])
df2 = df2.drop(columns=df.columns[-1], axis=1)
df2

,要員工数,設備工数,勤怠工数,勤怠-要員
01,0.0,0.0,0.0,0.0
02,0.0,0.0,0.0,0.0
03,0.0,0.0,0.0,0.0
04,0.0,0.0,0.0,0.0
05,0.0,0.0,0.0,0.0
06,0.0,0.0,0.0,0.0
07,0.0,0.0,0.0,0.0
08,8.5,4.0,8.5,0.0
09,8.5,4.0,8.5,0.0
10,8.0,4.0,8.0,0.0


In [ ]:
df2 = df.set_index('index')
df2 = df2.drop('合計', axis=1)
df2 = df2.drop(df2.index[-1])
df2 = df2.astype(float)
df2['Total'] = df2.sum(axis=1)
df2

In [42]:
df.iloc[4] = df2.iloc[0] - 4

ValueError: Must have equal len keys and value when setting with an iterable

In [25]:
# 作りたいテーブル
today = datetime.date.today()
# print(f'{today.year}年{today.month-1}月の開始曜日と日数 : {calendar.monthrange(today.year, today.month-1)}')

weekday, numDays = calendar.monthrange(2023, 5)

print([4]*numDays)
print([8-5]*numDays)
for row in range(weekday+1, weekday+6):
    row1 = [1 if (i+1) % 7 == row else 0 for i in range(31)]
    print(row1)


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [250]:
def extract_row_data(rowNum):
    rowData = []
    css = f'#divTblProductCostInfo > div > div > div > div > table > tbody > tr:nth-child({rowNum}) > td'
    for ele in (driver.find_elements(By.CSS_SELECTOR, css)):
        txt = ele.text
        if txt == '':
            rowData.append(None)
        else:
            rowData.append(txt)
    return rowData

In [251]:
# ズームコントロールがうまくいかない
#driver.execute_script("document.body.style.zoom='30%'")

In [252]:
table = []

for i in range(7):
    rowData = extract_row_data(i+1)
    print(len(rowData))
    table.append(rowData)

63
63
63
63
63
63
63


In [287]:
df = pd.DataFrame(table)

In [259]:
from datetime import date
import calendar

today = datetime.date.today()


(0, 31)


In [284]:
items =  df.iloc[:, 3:12]
items.head(1)

,3,4,5,6,7,8,9,10,11
0,KP01YRSX00002,最適化による製品設計自動化技術開発,最適化による製品設計自動化技術開発（71期）,要員・設備,解析,その他CAE,計画内作業(試験、作図等資料作成全般),CAD(h)_開発,None


In [288]:
productionCosts = df.iloc[:, 13:44].fillna(0.0).astype(float)
numOfDays = calendar.monthrange(today.year, today.month-1)
productionCosts.columns = [f'{i+1}日' for i in range(numOfDays[1])]
productionCosts.index = list(df.iloc[:, 3])
productionCosts.head(2)

,1日,2日,3日,4日,5日,6日,7日,8日,9日,10日,...,22日,23日,24日,25日,26日,27日,28日,29日,30日,31日
KP01YRSX00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,0.0,0.0,4.0,4.0,4.0
KP02YRSX00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,3.5,3.0,...,4.0,3.0,2.5,2.5,2.5,0.0,0.0,3.0,3.0,2.0


Actions action = new Actions(driver);
WebElement nametxt = driver.findElement(By.id("name"));
action.moveToElement(nametxt).click().sendKeys("Test Name").perform();

In [353]:
for positon, productionCost in enumerate(productionCosts.iloc[0]):
    # print(positon+14, productionCost)
    css = f'#divTblProductCostInfo > div > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child({positon+14})'
    ele = driver.find_element(By.CSS_SELECTOR, css)
    actionChains.click(ele).perform()
    actionChains.send_keys(Keys.BACKSPACE).perform()
    actionChains.send_keys(f'{productionCost}').perform()

In [346]:
for columns in range(14, 16):
    css = f'#divTblProductCostInfo > div > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child({columns})'
    ele = driver.find_element(By.CSS_SELECTOR, css)
    actionChains.click(ele).perform()
    actionChains.send_keys(Keys.BACKSPACE).perform()
    actionChains.send_keys('1').perform()